#  Discrete Choice Dynamic Programming
## A Dynamic Probit Model

So far, we were working in *static* problems when the agent has to decide between alternatives that do not involve a dynamic nature.
In this lecture, we are going to cover a very simple example of Discrete Choice Dynamic Programming were time is another *state variable*. Again and for simplification, the decision space is minimal so we can focus on the innovation with respect to previous lecture: time.

Assume an agent $n \in N$ has to decide between two alternatives, working $(i=1)$ or leisure $(i=0)$. The *current* utility can be writen as

$U_{n1}=\beta x_n+\varepsilon_{n1} $ if the agent chose to work

$U_{n0}=\mu +\varepsilon_{n0} $ if the agent chose to do not work, i.e., leisure.

However, the agent cares about future. Future is uncertain and there is no perfect forseight, so the agent has to form an *expectation*. The maximization problem of the agent can be thought as the maximization of current utility plus expected utility from tomorrow to the last day. The value function of this agent at time $t<T$ reads

$V(t,d;\varepsilon_t) =  max \{ U_{n1}-U_{n0} + EV_m(t+1,d;\varepsilon_{t+1}),EV_m(t+1,d+1;\varepsilon_{t+1})\}$

And for the last day, $t=T$

$V(T,d;\varepsilon_T) =  max \{ U_{n1}-U_{n0} +\gamma \pi(d)+ EV_m(1,0;\varepsilon_{t+1,m+1}),\gamma \pi(d+1)+EV_m(1,0;\varepsilon_{t+1,m+1})\}$

How do we solve this value function? *Backward iteration*.

At time t=T, the terms $EV_m(1,0;\varepsilon_{t+1,m+1})\}$ enters at both sides of the maximization bracket. 

Let's assume a month has a maximum of 5 days. The Prob is now a matrix

In [12]:
using Plots, Distributions, Random
using DataFrames, CSV
using Optim

## Set Parameters and Distributions
β=0.03;
pm=0; # Probability of being fired
P=3.0; # Non-pecunary cost
μ=4.0;
sim=100; # Number of simulated teachers
dist=Normal(); # Dist of error term
trdist(l)=truncated(dist,l, Inf); # Truncated Normal dist fn

In [2]:
payment(d,treat=1,M=10)=treat*(500+50*max(0,d-M))+(1-treat)*1000; # Payment schedule for treated and control

In [3]:
Tm=15;
ϵ_th=fill(NaN, Tm, Tm)

15×15 Matrix{Float64}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 N

For the last day, we could have worked zero days, one day, two days, three days ... T-1 days.
For zero dyas, the payment would be

In [4]:
ϵ_th[Tm,1]=-μ+P+β*(payment(1)-payment(0))

-1.0

If we extend the number of days to 15 days, we will have

In [5]:
Tm=15;
ϵ_th=fill(NaN, Tm, Tm)
for d=0:Tm-1
    ϵ_th[Tm,d+1]=-μ+P+β*(payment(d+1)-payment(d))
end
ϵ_th[end,:]

15-element Vector{Float64}:
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
 -1.0
  0.5
  0.5
  0.5
  0.5
  0.5

Once we computed the thresholds for the last day, we can assign probabilities.

In [6]:
Pr=fill(NaN, Tm, Tm)
ϵ_th=fill(NaN, Tm, Tm)
for d=0:Tm-1
    ϵ_th[Tm,d+1]=-μ+P+β*(payment(d+1)-payment(d));
    Pr[Tm,d+1]=cdf(dist,ϵ_th[Tm,d+1]);
end
Pr[end,:]

15-element Vector{Float64}:
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.15865525393145702
 0.6914624612740131
 0.6914624612740131
 0.6914624612740131
 0.6914624612740131
 0.6914624612740131

And finally, we can compute the value function at the last day

In [7]:
Pr=fill(NaN, Tm, Tm);
ϵ_th=fill(NaN, Tm, Tm);
EV=fill(NaN,Tm, Tm);
for d=0:Tm-1
    ϵ_th[Tm,d+1]=-μ+P+β*(payment(d+1)-payment(d));
    Pr[Tm,d+1]=cdf(dist,ϵ_th[Tm,d+1]);
    EV[Tm,d+1]=(1-Pr[Tm,d+1])*(μ-P+β*payment(d)+mean(trdist(ϵ_th[Tm,d+1])))+Pr[Tm,d+1]*(β*payment(d+1))
end
EV[end,:]

15-element Vector{Float64}:
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.08331547058769
 16.697796557401304
 18.197796557401304
 19.697796557401304
 21.197796557401304
 22.697796557401304

In [8]:
function probs(β::Float64,μ::Float64) # Probability of working
    Pr=fill(NaN, Tm, Tm); #Probability matrix for L=0, work
    ϵ_th=fill(NaN, Tm, Tm); # Threshold matrix for L=0, work
    EV=fill(NaN, Tm, Tm); # Value funtion matrix
        for t=Tm:-1:1
            if t==Tm # Last day
                for d=0:Tm-1
                    ϵ_th[Tm,d+1]=-μ+P+β*(payment(d+1)-payment(d));
                    Pr[Tm,d+1]=cdf(dist,ϵ_th[Tm,d+1]);
                    EV[Tm,d+1]=(1-Pr[Tm,d+1])*(μ-P+β*payment(d)+mean(trdist(ϵ_th[Tm,d+1])))+
                    Pr[Tm,d+1]*(β*payment(d+1))
                end
            else # From day 1 to Tm-1
                for d=0:t-1
                    ϵ_th[t,d+1]=-μ+P+EV[t+1,d+2]-EV[t+1,d+1]
                    Pr[t,d+1]=cdf(dist,ϵ_th[t,d+1])
                    EV[t,d+1]=(1-Pr[t,d+1])*(μ-P+mean(trdist(ϵ_th[t,d+1]))+EV[t+1,d+1])+
                    Pr[t,d+1]*(EV[t+1,d+2])
                end
            end
        end
    return Pr,EV
end

probs (generic function with 1 method)

In [9]:
probs(β,μ)[1]

15×15 Matrix{Float64}:
 0.158677  NaN         NaN         …  NaN         NaN         NaN
 0.158661    0.158764  NaN            NaN         NaN         NaN
 0.158656    0.158684    0.159187     NaN         NaN         NaN
 0.158655    0.158662    0.158804     NaN         NaN         NaN
 0.158655    0.158656    0.158691     NaN         NaN         NaN
 0.158655    0.158655    0.158662  …  NaN         NaN         NaN
 0.158655    0.158655    0.158656     NaN         NaN         NaN
 0.158655    0.158655    0.158655     NaN         NaN         NaN
 0.158655    0.158655    0.158655     NaN         NaN         NaN
 0.158655    0.158655    0.158655     NaN         NaN         NaN
 0.158655    0.158655    0.158655  …  NaN         NaN         NaN
 0.158655    0.158655    0.158655     NaN         NaN         NaN
 0.158655    0.158655    0.158655       0.691462  NaN         NaN
 0.158655    0.158655    0.158655       0.691462    0.691462  NaN
 0.158655    0.158655    0.158655       0.691462    0

Now, let's simulate some data for this problem.

In [10]:
function simulate_data()
    EV=probs(β,μ)[2];
    L_mat =fill(NaN, Tm, sim)
    #L_mat =fill(NaN, Tm+1, sim); # Matrix to store the leisure decision at the individual level
    D_plus_mat = fill(NaN, Tm,sim); # Matrix to store # days worked up to day t, at the individual level
    D_mat = fill(NaN, Tm+1,sim)
    D_mat[1,:].=0
    eps = fill(NaN,Tm,sim)
    for id=1:sim
        d=1 # Should be d=0, but the index starts at 1.
        for t=1:Tm
            ϵ_shock=rand(dist)
            if t<Tm
                lhs=μ-P+ϵ_shock+EV[t+1,d]
                rhs=EV[t+1,d+1]
                if lhs>rhs
                    dec=1 # Decision: 1 do not work, 0 going to work
                else
                    dec=0
                end
                d=d+(1-dec)
            else
                #lhs=μ-P+ϵ_shock+β*payment(d-1,treat) # Because the function use d ∈ [0,Tm-1]
                lhs=ϵ_shock+β*payment(d-1) # Because the function use d ∈ [0,Tm-1]

                rhs=β*payment(d) # Because the function use d ∈ [0,Tm-1]
                if lhs>rhs
                    dec=1
                else
                    dec=0
                end
                d=d+(1-dec)
            end
            eps[t,id]=ϵ_shock
            L_mat[t,id]=dec
            D_plus_mat[t,id]=d-1 # Should be d=0, but the index starts at 1.
            D_mat[t+1,id]=D_plus_mat[t,id]
    
        end
    end
    D_mat=D_mat[1:Tm,:]
    #L_mat=L_mat[1:Tm,:]
    return L_mat,D_plus_mat, D_mat,eps
end

simulate_data (generic function with 1 method)

In [11]:
simulate_data()[1]

15×100 Matrix{Float64}:
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  …  1.0  0.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0     0.0  1.0  1.0  0.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  1.0  1.0  1.0  1.0  0.0  1.0
 0.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  1.0  0.0  1.0  1.0  1.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  0.0  1.0  1.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0     1.0  0.0  1.0  1.0  1.0  1.0  0.0
 1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0     1.0  1.0  0.0  1.0  1.0  1.0  1.0
 1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  0.0  1.0  1.0
 0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0 

Finally, we can write our Log-Likelihood function as

In [14]:
function logL_fn(θ) #LogL function
    β=θ[1]
    μ=θ[2]
    Pr=probs(β,μ)[1];
    logL=0
    #n=nrow(data)
    n=100
    for id=1:n # I cannot vectorize bc of `Pr`
        for t=1:Tm
            d=Int(data[id,4]);
            t=Int(data[id,2]);
            l=Int(data[id,5]);
            logL=logL+log(1-Pr[t,d+1])*l+log(Pr[t,d+1])*(1-l)
        end
    end
    return -(logL)
end

logL_fn (generic function with 1 method)